In [1]:
import $exec.spark_functionality


Compiling /home/jovyan/work/etl_textreuse/spark_functionality.scCompiling /home/jovyan/work/etl_textreuse/spark_functionality.sc #2

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark UI

import $exec.$                  


# 1. create piecewise reuses

Create version of the reuse data where there is a pieces table recording each piece in a reuse in each text, linked by a textreuses table.

In [2]:
val textreuses1 = get_a3s("textreuses")
// uncomment the below to test with a small sample
// val rdd = sc.parallelize(textreuses1.take(1000))
// val rddDf = spark.createDataFrame(rdd,schema=textreuses1.schema)
// val textreuses = materialise_local("textreuses",df=rddDf)

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

textreuses1: DataFrame = [textreuse_id: bigint, trs1_id: bigint ... 7 more fields]

In [9]:
val pieces = materialise_row_numbers(
    "orig_pieces", 
    spark.sql("""
    SELECT * FROM (
    SELECT  
        trs1_id AS trs_id,
        trs1_start AS trs_start,
        trs1_end AS trs_end 
    FROM textreuses
    UNION
    SELECT
        trs2_id AS trs_id,
        trs2_start AS trs_start,
        trs2_end AS trs_end 
    FROM textreuses
    )
    ORDER BY trs_id,trs_start,trs_end
    """),
    "piece_id"
)

0 / 1

In [10]:
val orig_textreuses = materialise_a3s_if_not_exists(
    "orig_textreuses",
    spark.sql("""
    SELECT 
        t.textreuse_id, 
        op1.piece_id AS piece1_id, 
        op2.piece_id AS piece2_id,
        t.align_length,
        t.positives_percent 
    FROM textreuses t
    LEFT JOIN orig_pieces op1 ON t.trs1_id = op1.trs_id AND t.trs1_start=op1.trs_start AND t.trs1_end=op1.trs_end
    LEFT JOIN orig_pieces op2 ON t.trs2_id = op2.trs_id AND t.trs2_start=op2.trs_start AND t.trs2_end=op2.trs_end
"""))

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

orig_textreuses: DataFrame = [textreuse_id: bigint, piece1_id: bigint ... 3 more fields]

# 2. Defragment overlapping pieces to one id

Create an initial defragmentation of the piece_ids by mapping all pieces that overlap sufficiently in a given text document to one id

In [11]:
import spark.implicits._
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, ArrayType}
import org.apache.spark.sql.catalyst.encoders.RowEncoder

val input_type = StructType(Seq(
    StructField("trs_start",IntegerType),
    StructField("trs_end",IntegerType),
    StructField("piece_id",IntegerType)
))

import scala.collection.mutable.ArrayBuffer
import org.apache.spark.sql.expressions.Aggregator

object GetPieceIdMapping extends Aggregator[Row, ArrayBuffer[(Int,Int,Int)], Int] {
  import org.apache.spark.sql.catalyst.encoders.ExpressionEncoder

  // A zero value for this aggregation. Should satisfy the property that any b + zero = b
  def zero: ArrayBuffer[(Int,Int,Int)] = ArrayBuffer()
  // Combine two values to produce a new value. For performance, the function may modify `buffer`
  // and return it instead of constructing a new object
  def reduce(to_check: ArrayBuffer[(Int,Int,Int)], row: Row): ArrayBuffer[(Int,Int,Int)] = {
      val t_start = row.getInt(0)
      // check existing buffer for the last piece that is 180 characters 
      // before the current start offset
      val idx = to_check.indexWhere(_._1 >= t_start - 180)
      // if 0 then clear the buffer
      // otherwise remove unsuitable pieces
      if (idx < 0) to_check.clear() else to_check.remove(0, idx)
      // add the current piece to the end of buffer
      to_check += ((t_start,row.getInt(1),row.getInt(2)))
      to_check
  }
  // Merge two intermediate values
  def merge(b1: ArrayBuffer[(Int,Int,Int)], b2: ArrayBuffer[(Int,Int,Int)]): ArrayBuffer[(Int,Int,Int)] = {
//      b1 ++= b2 & sort
      throw new UnsupportedOperationException()
  }
  // Transform the output of the reduction
  def finish(to_check: ArrayBuffer[(Int,Int,Int)]): Int = {
      val (t_start, t_end, piece_id) = to_check.last
      to_check.find(r => {
          val limit = math.min(
              math.max(
                  math.min(t_end - t_start, r._2 - r._1)
                  /4, 10),
              180)
          math.abs(r._1 - t_start) <= limit && math.abs(r._2 - t_end) <= limit
      }).get._3 // get the id of the first piece that is valid
  }
  // Specifies the Encoder for the intermediate value type
  def bufferEncoder: Encoder[ArrayBuffer[(Int,Int,Int)]] = ExpressionEncoder()
  // Specifies the Encoder for the final output value type
  def outputEncoder: Encoder[Int] = Encoders.scalaInt
}

// Register the function to access it
spark.udf.register("get_piece_id_mapping", functions.udaf(GetPieceIdMapping,RowEncoder(input_type)))

import spark.implicits._

import org.apache.spark.sql.types.{StructType, StructField, IntegerType, ArrayType}

import org.apache.spark.sql.catalyst.encoders.RowEncoder


input_type: StructType = StructType(
  StructField("trs_start", IntegerType, true, {}),
  StructField("trs_end", IntegerType, true, {}),
  StructField("piece_id", IntegerType, true, {})
)
import scala.collection.mutable.ArrayBuffer

import org.apache.spark.sql.expressions.Aggregator


defined object GetPieceIdMapping
res10_7: expressions.UserDefinedFunction = UserDefinedAggregator(
  ammonite.$sess.cmd10$Helper$GetPieceIdMapping$@52d964a3,
  ExpressionEncoder(
    If(
      IsNull(
        BoundReference(0, ObjectType(interface org.apache.spark.sql.Row), true)
      ),
      Literal(
        null,
        StructType(
          StructField("trs_start", IntegerType, true, {}),
          StructField("trs_end", IntegerType, true, {}),
          StructField("piece_id", IntegerType, true, {})
        )
      ),
      CreateN

In [12]:
val piece_id_mappings = materialise_a3s_if_not_exists(
    "piece_id_mappings_tmp", 
    spark.sql("""
    SELECT 
        piece_id AS orig_piece_id,
        get_piece_id_mapping(trs_start,trs_end,piece_id) 
            OVER (PARTITION BY trs_id ORDER BY trs_start, piece_id) 
            AS defrag_mapping
    FROM orig_pieces
    """))

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

piece_id_mappings: DataFrame = [orig_piece_id: bigint, defrag_mapping: int]

We convert the mapping ids into monotonically increasing ids for new defragmented pieces

In [13]:
materialise_row_numbers(
  "defrag_id_mapping",
  spark.sql("""
    SELECT DISTINCT defrag_mapping
    FROM piece_id_mappings_tmp
    ORDER BY defrag_mapping
    """),
  "defrag_piece_id"
)

rdd at spark_functionality.sc:158 
 (kill)

0 / 25

0 / 34

0 / 34

0 / 33

0 / 34

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

Add the piece id mapping back to piece_id

In [14]:
val updated_piece_id_mappings = materialise_a3s("piece_id_mappings", spark.sql("""
SELECT orig_piece_id, defrag_piece_id FROM piece_id_mappings_tmp
INNER JOIN defrag_id_mapping USING(defrag_mapping)
"""))
// delete the temporary mapping dataframes
delete_s3("piece_id_mappings_tmp")
delete_s3("defrag_id_mapping")

parquet at spark_functionality.sc:120 
 (kill)

0 / 34

parquet at spark_functionality.sc:120 
 (kill)

0 / 34

parquet at spark_functionality.sc:120 
 (kill)

0 / 200

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

updated_piece_id_mappings: DataFrame = [orig_piece_id: bigint, defrag_piece_id: bigint]

## 3. Gather defragmented pieces and resulting edges

First gather the defragmented edges and their offset boundaries

In [15]:
val defrag_pieces = materialise_a3s("defrag_pieces", spark.sql("""
SELECT defrag_piece_id AS piece_id, trs_id, MIN(trs_start) AS trs_start, MAX(trs_end) AS trs_end
FROM piece_id_mappings
INNER JOIN orig_pieces ON piece_id = orig_piece_id
GROUP BY defrag_piece_id, trs_id
"""))

0 / 34

parquet at spark_functionality.sc:120 
 (kill)

0 / 43

parquet at spark_functionality.sc:120 
 (kill)

0 / 200

parquet at spark_functionality.sc:120 
 (kill)

0 / 200

parquet at spark_functionality.sc:116 
 (kill)

0 / 1

defrag_pieces: DataFrame = [piece_id: bigint, trs_id: bigint ... 2 more fields]

Map the reuses from the original pieces to the defragmeneted pieces

In [16]:
materialise_row_numbers(
    "defrag_textreuses",
    spark.sql("""
    SELECT
        pm1.defrag_piece_id AS piece1_id, 
        pm2.defrag_piece_id AS piece2_id,
        COUNT(*) AS num_orig_links
    FROM orig_textreuses tr
    LEFT JOIN piece_id_mappings pm1 ON (tr.piece1_id=pm1.orig_piece_id)
    LEFT JOIN piece_id_mappings pm2 ON (tr.piece2_id=pm2.orig_piece_id)
    GROUP BY pm1.defrag_piece_id,pm2.defrag_piece_id
    ORDER BY piece1_id,piece2_id
    """),
    "textreuse_id"
)

rdd at spark_functionality.sc:158 
 (kill)

0 / 100

rdd at spark_functionality.sc:158 
 (kill)

0 / 34

rdd at spark_functionality.sc:158 
 (kill)

...

0 / 200

0 / 199

parquet at spark_functionality.sc:120 
 (kill)

0 / 200

0 / 1

# 4. Create adjacency list for clustering

Create a parquet file containing the defragmented textreuse network in the the form of an adjacency list. Also ensure that all nodes (i.e., defragmented pieces) are 0-indexed. Therefore, each array index contains the indices of other nodes it links to. Also coalese into a single partition for the clustering code.

In [17]:
val defrag_textreuses = get_a3s("defrag_textreuses")

materialise_a3s(
    "defrag_graph_adj_list",
    spark.sql("""
    WITH undirected_edges AS (
      SELECT piece1_id-1 as piece1_id, piece2_id-1 as piece2_id FROM defrag_textreuses
      UNION ALL
      SELECT piece2_id-1 AS piece1_id, piece1_id-1 AS piece2_id FROM defrag_textreuses
    )
    SELECT /*+ REPARTITION(1)*/ collect_list(piece2_id) AS edges
    FROM undirected_edges
    GROUP BY piece1_id
    ORDER BY piece1_id
    """))

0 / 1

0 / 82

# 5. clustering 

Done separately with the Go versionn of the Chinese Whispers algorithm